In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [9]:
# Create basic edges and nodes in the computational graph: 

a = tf.constant(5.6, shape =(),  dtype= tf.float32)
b = tf.constant(3.4, shape= (),  dtype= tf.float32)
c = a + b

d = tf.Variable(2.0, name="W")
d.assign(a*b)

print (a)
print (b)
print (c)
print (d)


Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Const_3:0", shape=(), dtype=float32)
Tensor("add_1:0", shape=(), dtype=float32)
<tf.Variable 'W_1:0' shape=() dtype=float32_ref>


In [10]:
# Learning to use TensorBoard: 
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

In [2]:
# Creating a simple Two-layer Neural net:

# Create a placeholder for the input to the layer and also for the ground truth labels: 
x = tf.placeholder(dtype= tf.float32, shape=[None, 32], name="x")   # x is a batch of 32x1 vectors. Size of batch is decide by variables fed into placeholder
y = tf.placeholder(dtype = tf.int32, shape=[None])

# Create variables for the weights and biases for the first layer: 
W1 = tf.get_variable("W1", shape=[32,16])
b1 = tf.get_variable("b1", shape= [16])

# Create variables for the second affine layer: 
W2 = tf.get_variable("W2", shape=[16,3])
b2 = tf.get_variable("b2", shape= [3])


        

In [6]:
# Create the operation for the first affine layer: 
a1 = tf.matmul(x,W1) + b1

# Create the operation for the ReLu layer: 
a2 = tf.nn.relu(a1)

# Create the operation for the second affine layer:
a3 = tf.matmul(a2,W2) + b2

# Create an operation to compute the loss: 
loss = tf.losses.hinge_loss(tf.one_hot(y,3), logits = a3)

# Create the optimizer and its update operations: 
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train_step = optimizer.minimize(loss)

nIter = 5

# Running my first tf Session: 
with tf.Session() as sess: 
    
    # Learning to use TensorBoard: 
    writer = tf.summary.FileWriter('.')
    writer.add_graph(tf.get_default_graph())
    
    # Initialize the variables:
    init = tf.global_variables_initializer()
    sess.run(init)
    

    # Run the session:
    for i in range(nIter):
        
        feed_dict = {x: np.random.randn(64,32) , y: np.random.randint(0,3 , size=64)}
        
        sess.run(train_step, feed_dict = feed_dict)
        
        print (" Iteration # ", i )
        
    
    writer.close()

 Iteration #  0
 Iteration #  1
 Iteration #  2
 Iteration #  3
 Iteration #  4


In [8]:
# MNIST Dataset learnt using a Convolutional Neural Network: 

# Step 1: Load in the training data: 
# Load training data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)

# Load Eval data:
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

# Print the shape of the data: 
print (" Train Data ", train_data.shape)
print (" Training labels ", train_labels.shape)

print (" Eval Data ", eval_data.shape)
print (" Eval labels ", eval_labels.shape)

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-u

In [23]:
# Create the model architecture: 

def cnn_model_fn(features, labels, mode): 
    
    """
    Description: 
    
    Inputs: 
    
    Outputs:
    
    
    """
    
    # Input Layer:
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    # 1st convolutional layer: Returns an output Tensor:
    conv1 = tf.layers.conv2d(inputs =input_layer, filters = 32, kernel_size = 5 , padding="same", activation=tf.nn.relu)
    
    # 1st Pooling layer: 
    pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size =2, strides=2)
    
    # 2nd convolutional layer: 
    conv2 = tf.layers.conv2d(inputs = pool1, filters = 64, kernel_size=5, padding="same" , activation= tf.nn.relu)
    
    # 2nd pooling layer: 
    pool2 = tf.layers.max_pooling2d(inputs = conv2 , pool_size=2, strides=2)
    
    # 1st dense layer with 1024 hidden units: 
#     [N,H,W,C] = tf.shape(pool2)    
    pool2_flat = tf.reshape(pool2, [-1,7*7*64])
    affine1 = tf.layers.dense(inputs = pool2_flat, units=1024, activation = tf.nn.relu)
    dropout1 = tf.layers.dropout(inputs = affine1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # 2nd dense layer that produces scores: 
    scores = tf.layers.dense(inputs = dropout1 , units = 10)
    
    # Not sure exactly what this is!
    predictions = {
    # Generate predictions (for PREDICT and EVAL mode)
    "classes": tf.argmax(input=scores, axis=1),
    # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
    # `logging_hook`.
    "probabilities": tf.nn.softmax(scores, name="softmax_tensor")
      }
    
    # Compute the predictions if in Prection mode
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=scores)
    
    # Configure the Training Op (for TRAIN mode): Using Stochastic Gradient Descent:
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [ ]:
# Create the Estimator: Requires an Estimator Spec as an input:
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir=".")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)

# Train the model
# The function to feed values to the inputs of the model:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# The train function is a wrapper function which runs the session and applies the optimization routine:
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=2000)

    #hooks=[logging_hook])


INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_model_dir': '.', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbfba197048>, '_save_checkpoints_steps': None, '_is_chief': True, '_train_distribute': None, '_save_summary_steps': 100, '_session_config': None, '_master': '', '_task_type': 'worker', '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_service': None}


INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_model_dir': '.', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbfba197048>, '_save_checkpoints_steps': None, '_is_chief': True, '_train_distribute': None, '_save_summary_steps': 100, '_session_config': None, '_master': '', '_task_type': 'worker', '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_service': None}


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:step = 401, loss = 2.24879 (37.090 sec)


INFO:tensorflow:global_step/sec: 2.29918


INFO:tensorflow:global_step/sec: 2.29918


INFO:tensorflow:step = 501, loss = 2.24289 (43.496 sec)


INFO:tensorflow:step = 501, loss = 2.24289 (43.496 sec)


In [25]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)


# Run the evaluate function of the estimator:
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2018-05-13-02:08:23


...

INFO:tensorflow:Finished evaluation at 2018-05-13-02:08:35


INFO:tensorflow:Finished evaluation at 2018-05-13-02:08:35


INFO:tensorflow:Saving dict for global step 200: accuracy = 0.1981, global_step = 200, loss = 2.27896


INFO:tensorflow:Saving dict for global step 200: accuracy = 0.1981, global_step = 200, loss = 2.27896


{'accuracy': 0.1981, 'loss': 2.2789571, 'global_step': 200}
